In [8]:
import datetime
from hashlib import sha256

MAX_ENTRIES_AMOUNT = 5
def isCryptographicPuzzleSolved(aBlock):
    return aBlock.hash() < (2**256) / aBlock.header['difficulty'] - 1
    
class Block: 
    def __init__(self, entries):
        self.header = {
            'prev_hash': 0,
            'nonce': 0,
            'timestamp': None,
            'entries_amount': len(entries),
            'difficulty': 1
        }
        if (len(entries) <= MAX_ENTRIES_AMOUNT):
            self.entries = entries
        else:
            raise 'Exceeding max block size'
            
    def hash(self):
        return int(sha256(repr(self.header).encode('utf-8') + repr(self.entries).encode('utf-8')).hexdigest(), 16)
        
    def __str__(self):
        entries = ",".join(self.entries)
        return """
        'block_hash': {0}
        
        'header': {{
            'prev_hash':{1}
            'nonce': {2}
            'timestamp': {3}
            'entries_amount': {4}
            'difficulty': {5}
        }}
        
        'entries': [
            {6}
        ]
        """.format(hex(self.hash()), hex(self.header['prev_hash']), self.header['nonce'], self.header['timestamp'], self.header['entries_amount'], self.header['difficulty'], entries)
    
class Blockchain:
    def __init__(self):
        self.blocks = []
        self.last_block_hash = 0
        
    def addBlock(self, newBlock):
        if (self.isBlockValid(newBlock)):
            self.blocks.append(newBlock)
            self.last_block_hash = newBlock.hash()
            return True
        return False
    
    def isBlockValid(self, block):
        return block.header['prev_hash'] == self.last_block_hash and isCryptographicPuzzleSolved(block)
    
    def getLastHash(self):
        return self.last_block_hash
    
    def printBlockChain(self):
        for block in self.blocks:
            print ('-----------------------------------------------------------------------------')
            print (block)
            print ('-----------------------------------------------------------------------------')

In [9]:
def mine(prev_hash, block):
    block.header['prev_hash'] = prev_hash
    block.header['timestamp'] = datetime.datetime.now()
    while not isCryptographicPuzzleSolved(block):
        block.header['nonce'] += 1
        block.header['timestamp'] = datetime.datetime.now()

In [17]:
# Example creating a blockchain and adding one block only. 
# The second block doesn't get added because it does not match the prev_hash
coolBlockChain = Blockchain()
newBlock = Block(["{'user_id': 'user_1', 'user_data': 'data_1'}", "{'user_id': 'user_2', 'user_data': 'data_2'}"])

mine(coolBlockChain.getLastHash(), newBlock)

coolBlockChain.addBlock(newBlock)
coolBlockChain.addBlock(newBlock)
coolBlockChain.printBlockChain()

-----------------------------------------------------------------------------

        'block_hash': 0xfb2f190c5f1e0b1e9baf855e840226d65c37a96948367e3629161cb7499ba42
        
        'header': {
            'prev_hash':0x0
            'nonce': 0
            'timestamp': 2021-04-27 20:11:22.180968
            'entries_amount': 2
            'difficulty': 1
        }
        
        'entries': [
            {'user_id': 'user_1', 'user_data': 'data_1'},{'user_id': 'user_2', 'user_data': 'data_2'}
        ]
        
-----------------------------------------------------------------------------


In [18]:
def adjustDifficulty(prevDifficulty, elapsedTime, targetTime, blocksProcessed):
    return prevDifficulty * (blocksProcessed/elapsedTime)*targetTime

In [19]:
# Example of how difficulty is adjusted.
# Simulation of a running blockchain ecosystem.

def generateBlock(blockchain, difficulty, i):
    newBlock = Block(["{{'user_id': 'user_{0}', 'user_data': 'data_{0}'}}".format(i)])
    newBlock.header['difficulty'] = difficulty
    mine(blockchain.getLastHash(), newBlock)
    blockchain.addBlock(newBlock)
    
    print ('{0} | Block {2} mined with difficulty: {1}!'.format(datetime.datetime.now(), difficulty, i))
    print (newBlock)
    
TARGET_TIME_IN_SECONDS = 12 # Every 60s a new block is mined in avg.
BLOCKS_PROCESSED_TO_ADJUST_DIFFICULTY = 256 # Every 256 blocks difficulty is adjusted to match TARGET_TIME.

elapsedTimeInSeconds = 0
blocksProccesed = 0

awesomeBlockchain = Blockchain()
difficulty = 1

startTime = datetime.datetime.now()

genesisBlock = Block(["{'user_id': 'user_genesis', 'user_data': 'data_genesis'}"])
mine(awesomeBlockchain.getLastHash(), genesisBlock)
awesomeBlockchain.addBlock(genesisBlock)

blocksProccesed += 1
i = 0
# Simulation of a working blockchain.
# A random block is generated in each iteration and difficulty is adjusted dynamically
while len(awesomeBlockchain.blocks) < 10000:
    if blocksProccesed >= BLOCKS_PROCESSED_TO_ADJUST_DIFFICULTY:
        elapsedTimeInSeconds = (datetime.datetime.now() - startTime).total_seconds()
        difficulty = adjustDifficulty(difficulty, elapsedTimeInSeconds, TARGET_TIME_IN_SECONDS, blocksProccesed)
        blocksProccesed = 0
        startTime = datetime.datetime.now()
        
    generateBlock(awesomeBlockchain, difficulty, i)
    i += 1
    blocksProccesed += 1

2021-04-27 20:12:45.157030 | Block 0 mined with difficulty: 1!

        'block_hash': 0x34a0ecc691044bd5edd3eaf928b19f5fddd7fdf11148be210d3919f6c065b0e2
        
        'header': {
            'prev_hash':0x864f058798db4b5ae968fec5d339b359eef39030d3146e62e93ce50001b6287d
            'nonce': 0
            'timestamp': 2021-04-27 20:12:45.157030
            'entries_amount': 1
            'difficulty': 1
        }
        
        'entries': [
            {'user_id': 'user_0', 'user_data': 'data_0'}
        ]
        
2021-04-27 20:12:45.158031 | Block 1 mined with difficulty: 1!

        'block_hash': 0xee0566ce97bd8a6a23805e5ca071ca0dff76520e221b6c13b95d75d6f5dd5478
        
        'header': {
            'prev_hash':0x34a0ecc691044bd5edd3eaf928b19f5fddd7fdf11148be210d3919f6c065b0e2
            'nonce': 0
            'timestamp': 2021-04-27 20:12:45.158031
            'entries_amount': 1
            'difficulty': 1
        }
        
        'entries': [
            {'user_id': 'use

2021-04-27 20:12:45.938774 | Block 255 mined with difficulty: 24575.803393572853!

        'block_hash': 0x1b62ad3974c95e584fa8db57c74cbf3ca15dc915e13b7e16d984b7602daa8
        
        'header': {
            'prev_hash':0x7cc26a5a433994cdeb4a16d07da0e789c1e65002b0b7001701d590e9af3dde69
            'nonce': 40334
            'timestamp': 2021-04-27 20:12:45.938774
            'entries_amount': 1
            'difficulty': 24575.803393572853
        }
        
        'entries': [
            {'user_id': 'user_255', 'user_data': 'data_255'}
        ]
        
2021-04-27 20:12:45.992774 | Block 256 mined with difficulty: 24575.803393572853!

        'block_hash': 0x15990a5762252449825c0879b0fd54632a1c9cc9ab1df94c52e325f5570ba
        
        'header': {
            'prev_hash':0x1b62ad3974c95e584fa8db57c74cbf3ca15dc915e13b7e16d984b7602daa8
            'nonce': 2143
            'timestamp': 2021-04-27 20:12:45.992774
            'entries_amount': 1
            'difficulty': 24575.8033935

2021-04-27 20:12:55.438423 | Block 270 mined with difficulty: 24575.803393572853!

        'block_hash': 0xc430f52aa14655bdf9d742904be9757c194599d960be656cf233468e6060
        
        'header': {
            'prev_hash':0x93410e2104999ce09c51b9a6b30ab0d2f636a6bbb2af1f5e5d034646e9a3
            'nonce': 41823
            'timestamp': 2021-04-27 20:12:55.438423
            'entries_amount': 1
            'difficulty': 24575.803393572853
        }
        
        'entries': [
            {'user_id': 'user_270', 'user_data': 'data_270'}
        ]
        
2021-04-27 20:12:55.723277 | Block 271 mined with difficulty: 24575.803393572853!

        'block_hash': 0xc9267aada71a5d919c44c497c49766cb3d152d51741e4dead535df625abc
        
        'header': {
            'prev_hash':0xc430f52aa14655bdf9d742904be9757c194599d960be656cf233468e6060
            'nonce': 21501
            'timestamp': 2021-04-27 20:12:55.723277
            'entries_amount': 1
            'difficulty': 24575.803393572853


2021-04-27 20:12:59.732373 | Block 286 mined with difficulty: 24575.803393572853!

        'block_hash': 0x28d780674e05ccc4ded4a51487f736cb198f9191a2ad3084b041e3a4088e0
        
        'header': {
            'prev_hash':0x24af796c54b78f2ce43d181800281d6fe5a53fde2517dd77b8ff83fedd5aa
            'nonce': 26303
            'timestamp': 2021-04-27 20:12:59.732373
            'entries_amount': 1
            'difficulty': 24575.803393572853
        }
        
        'entries': [
            {'user_id': 'user_286', 'user_data': 'data_286'}
        ]
        
2021-04-27 20:13:00.501998 | Block 287 mined with difficulty: 24575.803393572853!

        'block_hash': 0xd4094729901d921ff99c902208b1774d45cec333e6d1a950c3a06fb9ff4e
        
        'header': {
            'prev_hash':0x28d780674e05ccc4ded4a51487f736cb198f9191a2ad3084b041e3a4088e0
            'nonce': 40093
            'timestamp': 2021-04-27 20:13:00.501998
            'entries_amount': 1
            'difficulty': 24575.8033935728

2021-04-27 20:13:06.698883 | Block 302 mined with difficulty: 24575.803393572853!

        'block_hash': 0x22756c3f5cee4b5dac455c63988ca86d78dbad6f2554a71e717a1c50404a0
        
        'header': {
            'prev_hash':0x2a4715eef0f8f9c516a35147e4505543d270b32a670c3cb78bf9d8cde1b11
            'nonce': 37330
            'timestamp': 2021-04-27 20:13:06.698883
            'entries_amount': 1
            'difficulty': 24575.803393572853
        }
        
        'entries': [
            {'user_id': 'user_302', 'user_data': 'data_302'}
        ]
        
2021-04-27 20:13:06.781881 | Block 303 mined with difficulty: 24575.803393572853!

        'block_hash': 0x2ee000e904450787db184e92a7eb7d008ec236aacb2d0f4887332907ff11
        
        'header': {
            'prev_hash':0x22756c3f5cee4b5dac455c63988ca86d78dbad6f2554a71e717a1c50404a0
            'nonce': 6001
            'timestamp': 2021-04-27 20:13:06.781881
            'entries_amount': 1
            'difficulty': 24575.80339357285

...

2021-04-27 20:55:35.937349 | Block 795 mined with difficulty: 1154890.0582254436!

        'block_hash': 0x4be8c68cc7580b5c5fd12e7d8a5084c874a2b63a250b628e1feea11619d
        
        'header': {
            'prev_hash':0xa403a4c4ed73955d6673702c75ef6e6b1c55be27e6fbdc78e82bdf61dbe
            'nonce': 3940911
            'timestamp': 2021-04-27 20:55:35.937349
            'entries_amount': 1
            'difficulty': 1154890.0582254436
        }
        
        'entries': [
            {'user_id': 'user_795', 'user_data': 'data_795'}
        ]
        
2021-04-27 20:56:01.187687 | Block 796 mined with difficulty: 1154890.0582254436!

        'block_hash': 0x6fa8b8478990d03136ab69b1f9a65dc3ad5dcc48ed190d947d246a55e3e
        
        'header': {
            'prev_hash':0x4be8c68cc7580b5c5fd12e7d8a5084c874a2b63a250b628e1feea11619d
            'nonce': 2149278
            'timestamp': 2021-04-27 20:56:01.187687
            'entries_amount': 1
            'difficulty': 1154890.0582254436


2021-04-27 20:59:07.250053 | Block 810 mined with difficulty: 1154890.0582254436!

        'block_hash': 0x9908088aaacd21b39dbf6c03f57ecd4328478ae68d562c4a540f05472c3
        
        'header': {
            'prev_hash':0x4a84d151c97f21af52684806222a010e7b856875f99abfe36f34529b2f4
            'nonce': 1810552
            'timestamp': 2021-04-27 20:59:07.250053
            'entries_amount': 1
            'difficulty': 1154890.0582254436
        }
        
        'entries': [
            {'user_id': 'user_810', 'user_data': 'data_810'}
        ]
        
2021-04-27 20:59:27.545846 | Block 811 mined with difficulty: 1154890.0582254436!

        'block_hash': 0xcab6ab55d89ec011c8473f3e91dff0bb39a58b1f9a0049dfa058fc609da
        
        'header': {
            'prev_hash':0x9908088aaacd21b39dbf6c03f57ecd4328478ae68d562c4a540f05472c3
            'nonce': 1735868
            'timestamp': 2021-04-27 20:59:27.545846
            'entries_amount': 1
            'difficulty': 1154890.0582254436


2021-04-27 21:04:11.683951 | Block 825 mined with difficulty: 1154890.0582254436!

        'block_hash': 0x9ba792891523cdc3110ab0ad9752735e386105c4f9665884a5ef67f8b13
        
        'header': {
            'prev_hash':0x4b36e4d5d2f534881001d18a9aeb04ac1fadea54f5aea453e657bd5f4a4
            'nonce': 1239041
            'timestamp': 2021-04-27 21:04:11.683951
            'entries_amount': 1
            'difficulty': 1154890.0582254436
        }
        
        'entries': [
            {'user_id': 'user_825', 'user_data': 'data_825'}
        ]
        
2021-04-27 21:04:15.161190 | Block 826 mined with difficulty: 1154890.0582254436!

        'block_hash': 0x44e8b1f341d893f3b97a13ed4417b4eda051c00c7ad5b45fd6c37280c9a
        
        'header': {
            'prev_hash':0x9ba792891523cdc3110ab0ad9752735e386105c4f9665884a5ef67f8b13
            'nonce': 305311
            'timestamp': 2021-04-27 21:04:15.161190
            'entries_amount': 1
            'difficulty': 1154890.0582254436
 

2021-04-27 21:08:10.171571 | Block 840 mined with difficulty: 1154890.0582254436!

        'block_hash': 0x6fffad6c255f9c8a2763abc4e257de81c04a98c8a3c21ed6bbd639ea9c9
        
        'header': {
            'prev_hash':0x93f113cb59afc904066fc7533df34c031dd7f9960228523849e515f3776
            'nonce': 1599748
            'timestamp': 2021-04-27 21:08:10.171571
            'entries_amount': 1
            'difficulty': 1154890.0582254436
        }
        
        'entries': [
            {'user_id': 'user_840', 'user_data': 'data_840'}
        ]
        
2021-04-27 21:08:19.226289 | Block 841 mined with difficulty: 1154890.0582254436!

        'block_hash': 0xcdc83955a387aee10540b83dde87781e01944a41cc615d3092c1360903d
        
        'header': {
            'prev_hash':0x6fffad6c255f9c8a2763abc4e257de81c04a98c8a3c21ed6bbd639ea9c9
            'nonce': 761462
            'timestamp': 2021-04-27 21:08:19.226289
            'entries_amount': 1
            'difficulty': 1154890.0582254436
 

KeyboardInterrupt: 

In [15]:
# Example on how multiple clients issue transactions to the system and those are batched together in blocks with max size 5.

def receiveTransactionFromClient(i):
    return "{{'user_id': 'user_{0}', 'user_data': 'data_{0}'}}".format(i)

amazingBlockchain = Blockchain()
difficulty = 1

genesisBlock = Block([ "{'user_id': 'user_genesis', 'user_data': 'data_genesis'}"])
mine(amazingBlockchain.getLastHash(), genesisBlock)
amazingBlockchain.addBlock(genesisBlock)

# Receive 5 transactions and batch them together.
transactions = []
for i in range(5):
    transactions.append(receiveTransactionFromClient(i))

firstBlock = Block(transactions)
mine(amazingBlockchain.getLastHash(), firstBlock)
amazingBlockchain.addBlock(firstBlock)

transactions = []
# Receive 3 transactions and batch them together.
transactions = []
for i in range(3):
    transactions.append(receiveTransactionFromClient(i))

secondBlock = Block(transactions)
mine(amazingBlockchain.getLastHash(), secondBlock)
amazingBlockchain.addBlock(secondBlock)

amazingBlockchain.printBlockChain()

-----------------------------------------------------------------------------

        'block_hash': 0xf2c68d89fe9588e170814d1ac5155795370364f35da93917000141eaf9baa656
        
        'header': {
            'prev_hash':0x0
            'nonce': 0
            'timestamp': 2021-04-27 20:07:48.101896
            'entries_amount': 1
            'difficulty': 1
        }
        
        'entries': [
            transaction_genesis
        ]
        
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------

        'block_hash': 0x660ee41d96ee4aa8f8765cbd7b84e33fe25591d6dbab4dc4d77b1017a23b99fe
        
        'header': {
            'prev_hash':0xf2c68d89fe9588e170814d1ac5155795370364f35da93917000141eaf9baa656
            'nonce': 0
            'timestamp': 2021-04-27 20:07:48.101896
            'entries_amount': 5
            'difficulty': 1
        }
        
        'entries': [
        

In [16]:
# Example dispatching a block to be mine by multiple workers using multithreading module.
import threading
import copy

runningOutOfNamesBlockchain = Blockchain()

def mineConcurrent(prev_hash, block, blockchain, i):
        block.header['difficulty'] = 100000
        block.header['nonce'] = i**7
        block.header['prev_hash'] = prev_hash
        block.header['timestamp'] = datetime.datetime.now()
        while not isCryptographicPuzzleSolved(block):
                block.header['nonce'] += 1
                block.header['timestamp'] = datetime.datetime.now()
        if (blockchain.addBlock(block)):
            print ('Block mined by thread {0}'.format(i))

def dispatchBlock(W, block):
    threads = []
    block_mined = None
    last_hash = runningOutOfNamesBlockchain.getLastHash()
    for i in range(W):
        t = threading.Thread( target=mineConcurrent, args=(last_hash, copy.deepcopy(firstBlock), runningOutOfNamesBlockchain, i) )
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

# Receive 5 transactions and batch them together.
transactions = []
for i in range(5):
    transactions.append(receiveTransactionFromClient(i))

firstBlock = Block(transactions)

# Dispatch the block to 3 workers to mine the work.
dispatchBlock(3, firstBlock)

runningOutOfNamesBlockchain.printBlockChain()


Block mined by thread 2
-----------------------------------------------------------------------------

        'block_hash': 0xb69b5a5c3a958619c786e759f346663c709f891c57423c297479d128305
        
        'header': {
            'prev_hash':0x0
            'nonce': 2846
            'timestamp': 2021-04-27 20:07:49.287404
            'entries_amount': 5
            'difficulty': 100000
        }
        
        'entries': [
            {'user_id': 'user_0', 'user_data': 'data_0'},{'user_id': 'user_1', 'user_data': 'data_1'},{'user_id': 'user_2', 'user_data': 'data_2'},{'user_id': 'user_3', 'user_data': 'data_3'},{'user_id': 'user_4', 'user_data': 'data_4'}
        ]
        
-----------------------------------------------------------------------------
